# 🧠 MPL Trainer Examples
Three Python Scripts that train MLP Models with different data types, functions and results.

Scripts:
- MLP XOR Model Trainer.
- MLP Circle Dots Model Trainer.
- MLP Text Classifier Trainer.

## 🔀 MLP XOR Model Trainer

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# XOR Dataset
X = torch.tensor([[0,0], [0,1], [1,0], [1,1]], dtype=torch.float32)
Y = torch.tensor([[0], [1], [1], [0]], dtype=torch.float32)

# MLP Model Parameters
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden = nn.Linear(2, 4)  # 2 → 4 hidden neurons
        self.output = nn.Linear(4, 1)  # 4 → 1 output neuron
        self.relu = nn.ReLU()  # ReLU activation for hidden layer
        self.sigmoid = nn.Sigmoid()  # Sigmoid for output

    def forward(self, x):
        x = self.relu(self.hidden(x))  # Hidden layer with ReLU
        x = self.sigmoid(self.output(x))  # Output layer with Sigmoid
        return x

# Model Training
model = MLP()
loss_function = nn.BCELoss()  # Binary Cross Entropy Loss 
optimizer = optim.Adam(model.parameters(), lr=0.1)  # Learning rate

epochs = 5000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = loss_function(outputs, Y)
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Test Trained Model
print("\nTrained Outputs:")
predictions = model(X).detach()
rounded_predictions = torch.round(predictions)  # Round the predictions to 0 or 1

# Calculate accuracy
correct_predictions = (rounded_predictions.eq(Y)).sum()
accuracy = correct_predictions / Y.size(0) * 100
print(f"Accuracy: {accuracy.item()}%")

print(rounded_predictions)

## 🎯 MLP Circle Dots Model Trainer

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import matplotlib.pyplot as plt
import numpy as np

# Dataset
def generate_circle_data(num_points=100):
    # Generate random data points
    X = np.random.rand(num_points, 2)  # Random points in a 2D space
    y = np.array([1 if (x[0]**2 + x[1]**2 > 0.25) else 0 for x in X])  # Inside circle is 0, outside is 1
    return torch.tensor(X, dtype=torch.float32), torch.tensor(y, dtype=torch.float32).view(-1, 1)

X_train, Y_train = generate_circle_data(1000)  # Generate 1000 data points
X_test, Y_test = generate_circle_data(200)  # Generate 200 test points

# MLP Model Parameters
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden = nn.Linear(2, 4)
        self.output = nn.Linear(4, 1)
        self.relu = nn.ReLU()
        self.sigmoid = nn.Sigmoid()

    def forward(self, x):
        x = self.relu(self.hidden(x))
        x = self.sigmoid(self.output(x))
        return x

# Model Training
model = MLP()
loss_function = nn.BCELoss()  # Binary Cross Entropy Loss
optimizer = optim.Adam(model.parameters(), lr=0.1) # Learning rate

epochs = 5000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X_train)
    loss = loss_function(outputs, Y_train)
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
        print(f"Epoch {epoch}, Loss: {loss.item()}")

# Test Trained Model
with torch.no_grad():
    Y_pred = model(X_test).round()  # Round predictions to get 0 or 1
    accuracy = (Y_pred == Y_test).sum().item() / Y_test.size(0)
    print(f"\nAccuracy: {accuracy * 100:.2f}%")

    inside_count = (Y_pred == 0).sum().item()  # Count for points inside the circle (label 0)
    outside_count = (Y_pred == 1).sum().item()  # Count for points outside the circle (label 1)

    print(f"Points inside the circle: {inside_count}")
    print(f"Points outside the circle: {outside_count}")

# Plot the Data (Visualization)
plt.scatter(X_test[:, 0], X_test[:, 1], c=Y_pred.squeeze(), cmap='coolwarm', marker='o', alpha=0.6)
plt.title("Pattern Classification (Circle vs Outside)")
plt.xlabel("X1")
plt.ylabel("X2")
plt.show()


## 🔠 MLP Text Classifier Trainer

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

# Dataset
word_to_vec = {
    'apple': [1, 0, 0, 0],    # Elements of Group 0 (7)
    'banana': [0, 1, 0, 0],
    'cherry': [0, 0, 1, 0],
    'date': [0, 0, 0, 1],
    'orange': [1, 0, 0, 0],
    'grape': [0, 1, 0, 0],
    'pear': [0, 0, 1, 0],
    'dog': [0, 0, 0, 1],      # Elements of Group 1 (7)
    'car': [0, 0, 0, 1],
    'phone': [0, 0, 1, 0],
    'train': [1, 0, 0, 0],
    'ball': [0, 1, 0, 0],
    'computer': [1, 0, 0, 0],
    'table': [0, 1, 0, 0],
}

# Label Elements with their groups
X = torch.tensor(list(word_to_vec.values()), dtype=torch.float32)  # One-hot encoded vectors
Y = torch.tensor([[0], [0], [0], [0], [0], [0], [0], [1], [1], [1], [1], [1], [1], [1]], dtype=torch.long)  # Labels for CrossEntropyLoss

# MLP Model Parameters
class MLP(nn.Module):
    def __init__(self):
        super(MLP, self).__init__()
        self.hidden1 = nn.Linear(4, 8)  # Layers 4 input, 8 hidden
        self.hidden2 = nn.Linear(8, 16) # Added second hidden layer
        self.output = nn.Linear(16, 2)  # Output 2 units (binary)
        self.relu = nn.ReLU()

    def forward(self, x):
        x = self.relu(self.hidden1(x))
        x = self.relu(self.hidden2(x))  # Pass through second hidden layer
        x = self.output(x)
        return x

# Model Training
model = MLP()
loss_function = nn.CrossEntropyLoss()  # For multi-class classification
optimizer = optim.Adam(model.parameters(), lr=0.001)  # Learning rate

epochs = 5000
for epoch in range(epochs):
    optimizer.zero_grad()
    outputs = model(X)
    loss = loss_function(outputs, Y.squeeze())
    loss.backward()
    optimizer.step()

    if epoch % 500 == 0:
        predictions = torch.argmax(outputs, dim=1).tolist()
        print(f"Epoch {epoch}, Loss: {loss.item()}, Predictions: {predictions}")

# Calculate Accuracy
with torch.no_grad():
    outputs = model(X)
    predicted_classes = torch.argmax(outputs, dim=1)
    correct = (predicted_classes == Y.squeeze()).sum().item()
    accuracy = correct / len(Y) * 100
    print(f"Accuracy: {accuracy:.2f}%")

# Test Trained Model
def classify_word(word):
    if word in word_to_vec:
        vec = torch.tensor(word_to_vec[word], dtype=torch.float32).view(1, -1)
        with torch.no_grad():
            output = model(vec)
            predicted_class = torch.argmax(output, dim=1).item()
            print(f"Word: {word}, Group: {predicted_class}")
    else:
        print(f"Word '{word}' not found in the dictionary.")

# Example Usage
classify_word("apple")   # Should classify as group 0
classify_word("dog")     # Should classify as group 1
classify_word("car")     # Should classify as group 1
classify_word("cherry")  # Should classify as group 0
